<a href="https://colab.research.google.com/github/SMMubina/Swiggy-Restaurant-Recommendation/blob/main/Final_Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd

file= '/content/drive/MyDrive/Swiggy_Recommendation/swiggy.csv'
df= pd.read_csv(file)

In [ ]:
df.shape

(148541, 11)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148541 entries, 0 to 148540
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            148541 non-null  int64 
 1   name          148455 non-null  object
 2   city          148541 non-null  object
 3   rating        148455 non-null  object
 4   rating_count  148455 non-null  object
 5   cost          148410 non-null  object
 6   cuisine       148442 non-null  object
 7   lic_no        148312 non-null  object
 8   link          148541 non-null  object
 9   address       148455 non-null  object
 10  menu          148541 non-null  object
dtypes: int64(1), object(10)
memory usage: 12.5+ MB


In [ ]:
df.isnull().sum()

,0
id,0
name,86
city,0
rating,86
rating_count,86
cost,131
cuisine,99
lic_no,229
link,0
address,86


# Data Cleaning

* Mapping a Location as per need
* Handle missing values
* Drop unnecessary columns (lic_no, link, address, menu because they are not needed for recommendations).
* Convert cost to numeric.
* Process rating_count values
* Save the cleaned dataset as cleaned_data.csv.

In [ ]:
import pandas as pd

def fill_last_word(df, column_name):
    df[column_name] = df[column_name].str.split(',').str[-1].str.strip()
    return df

df_cleaned = fill_last_word(df, 'city')

In [ ]:
# Drop rows where essential fields are missing
df_cleaned = df_cleaned.dropna(subset=["name", "cuisine"])

In [ ]:
# Convert 'cost' column to numeric (remove ₹ and commas)
df_cleaned.loc[:, "cost"] = df_cleaned["cost"].str.replace("₹", "").str.replace(",", "").astype(float)

In [ ]:
# Fill missing 'cost' values with the median cost per 'city' and 'cuisine'
df_cleaned.loc[:, "cost"] = df_cleaned.groupby(["city", "cuisine"])["cost"].transform(lambda x: x.fillna(x.median()))

Streaming output truncated to the last 5000 lines.
<ipython-input-17-da377843ad21>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned.loc[:, "cost"] = df_cleaned.groupby(["city", "cuisine"])["cost"].transform(lambda x: x.fillna(x.median()))
<ipython-input-17-da377843ad21>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned.loc[:, "cost"] = df_cleaned.groupby(["city", "cuisine"])["cost"].transform(lambda x: x.fillna(x.median()))
<ipython-input-17-da377843ad21>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is

In [ ]:
df_cleaned.isnull().sum()

,0
id,0
name,0
city,0
rating,0
rating_count,0
cost,10
cuisine,0
lic_no,143
link,0
address,0


In [ ]:
# Display rows where 'cost' is NaN
print(df_cleaned[df_cleaned['cost'].isna()])

            id                                               name  \
24840   133551                                     Hotel Maithili   
24846   132123                                             Copisa   
24886   132957                              Barkha devi Ice cream   
25329   276257  M/S. TULASI ENTERPRISES(AMUL ICE CREAM ESSENTI...   
27884   272977                                               Amul   
30035   387716                                Froogal_Test_Outlet   
114751  316651                                        KANBRO EATS   
120099  167892                           Hotel Ashish Continental   
142842  304862                                       AAKASH FOODS   
143949  462679                                    MOMO MAGIC CAFE   

                  city rating     rating_count cost               cuisine  \
24840             Beed    3.8     100+ ratings  NaN  North Indian,Punjabi   
24846             Beed    3.2      20+ ratings  NaN        Pizzas,Italian   
24886    

In [ ]:
df_cleaned = df_cleaned.dropna(subset=["cost"])

In [ ]:
# Function to process 'rating_count' column
def convert_rating_count(val):
    if isinstance(val, str):
        val = val.lower().replace(" ratings", "").replace("+", "").strip()
        if "k" in val:  # Convert '1K' to 1000
            return int(float(val.replace("k", "")) * 1000)
        elif val.isdigit():
            return int(val)
    return 0  # Default for unknown cases

# Apply rating_count conversion
df_cleaned.loc[:, "rating_count"] = df_cleaned["rating_count"].apply(convert_rating_count)


In [ ]:
df_cleaned.loc[:, "rating"] = df_cleaned["rating"].replace("--", "0").astype(float)


In [ ]:
#drop lic_no, link, address, menu

df_cleaned = df_cleaned.drop(columns=['lic_no', 'link', 'address', 'menu'])


In [ ]:
df_cleaned.isnull().sum()

,0
id,0
name,0
city,0
rating,0
rating_count,0
cost,0
cuisine,0


In [ ]:
df_cleaned.to_csv('/content/drive/MyDrive/Swiggy_Recommendation/cleaned_data.csv', index=False)

print("✅ cleaned_data Files saved to Google Drive")

✅ cleaned_data Files saved to Google Drive


In [ ]:
import pandas as pd
df_cleaned = pd.read_csv('/content/drive/MyDrive/Swiggy_Recommendation/cleaned_data.csv')

In [ ]:
df_cleaned.head()

,id,name,city,rating,rating_count,cost,cuisine
0,567335,AB FOODS POINT,Abohar,0.0,0,200.0,"Beverages,Pizzas"
1,531342,Janta Sweet House,Abohar,4.4,50,200.0,"Sweets,Bakery"
2,158203,theka coffee desi,Abohar,3.8,100,100.0,Beverages
3,187912,Singh Hut,Abohar,3.7,20,250.0,"Fast Food,Indian"
4,543530,GRILL MASTERS,Abohar,0.0,0,250.0,"Italian-American,Fast Food"


# Data Preprocessing

* One-Hot Encode categorical variables (city & cuisine) so that they can be used in numerical computations.
* Save the One-Hot Encoder as encoder.pkl (so it can be reused in the Streamlit app).
* Save the preprocessed dataset as encoded_data.csv (ensuring the indices match cleaned_data.csv).

In [ ]:
import pandas as pd
import pickle
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder

# Load dataset
df_cleaned = pd.read_csv('/content/drive/MyDrive/Swiggy_Recommendation/cleaned_data.csv')

# Reset index to maintain alignment
df_cleaned.reset_index(drop=True, inplace=True)

# Select categorical and numerical features
categorical_features = ['city', 'cuisine']
numerical_features = ['rating', 'rating_count', 'cost']

# Apply One-Hot Encoding (Sparse format)
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)

# Fit and transform categorical data
encoded_sparse = encoder.fit_transform(df_cleaned[categorical_features])

# Convert numerical features to sparse format and stack
numerical_sparse = sparse.csr_matrix(df_cleaned[numerical_features].values)
final_sparse = sparse.hstack([numerical_sparse, encoded_sparse])

# Save the encoded sparse matrix in NPZ format (highly compressed)
sparse.save_npz("/content/drive/MyDrive/Swiggy_Recommendation/encoded_data_sparse.npz", final_sparse)

# ✅ Save both the encoder and categorical features
encoder_data = {
    "encoder": encoder,  # ✅ Store OneHotEncoder object
    "categorical_features": categorical_features  # ✅ Store feature names
}

with open("/content/drive/MyDrive/Swiggy_Recommendation/encoder.pkl", "wb") as file:
    pickle.dump(encoder_data, file)  # ✅ Save as dictionary

# 🚀 **Validation: Check if the indices still match**
loaded_sparse = sparse.load_npz("/content/drive/MyDrive/Swiggy_Recommendation/encoded_data_sparse.npz")

if df_cleaned.shape[0] == loaded_sparse.shape[0]:
    print("✅ Indices Match: cleaned_data.csv and encoded_data.csv are aligned.")
else:
    print("❌ Warning: Mismatch detected! Check indexing.")

print("✅ Encoding completed successfully using Sparse One-Hot Encoding (Memory Optimized).")

# ✅ Loading the encoder correctly in another script
with open("/content/drive/MyDrive/Swiggy_Recommendation/encoder.pkl", "rb") as file:
    encoder_data = pickle.load(file)
    encoder = encoder_data["encoder"]
    categorical_features = encoder_data["categorical_features"]


✅ Indices Match: cleaned_data.csv and encoded_data.csv are aligned.
✅ Encoding completed successfully using Sparse One-Hot Encoding (Memory Optimized).


In [ ]:
print("df_cleaned shape:", df_cleaned.shape)
print("Encoded sparse shape:", loaded_sparse.shape)


df_cleaned shape: (148432, 9)
Encoded sparse shape: (148432, 2687)


In [ ]:
import pandas as pd
from scipy import sparse

# Load cleaned data
df_cleaned = pd.read_csv('/content/drive/MyDrive/Swiggy_Recommendation/cleaned_data.csv')

# Load encoded sparse matrix
encoded_sparse = sparse.load_npz("/content/drive/MyDrive/Swiggy_Recommendation/encoded_data_sparse.npz")

# Check row count consistency
if df_cleaned.shape[0] == encoded_sparse.shape[0]:
    print("✅ Row count matches! The indices are aligned.")
else:
    print(f"❌ Mismatch detected! df_cleaned has {df_cleaned.shape[0]} rows, encoded data has {encoded_sparse.shape[0]} rows.")


✅ Row count matches! The indices are aligned.


In [ ]:
df_cleaned.reset_index(drop=False, inplace=True)  # Keep the original index
df_cleaned.to_csv('/content/drive/MyDrive/Swiggy_Recommendation/cleaned_data.csv', index=False)


In [ ]:
row_idx = 12  # Choose any row

# Check the original row
print("Original Data (cleaned_data.csv):")
print(df_cleaned.iloc[row_idx])

# Check the corresponding encoded row
print("\nEncoded Data Shape:", encoded_sparse[row_idx].shape)


Original Data (cleaned_data.csv):
level_0                    12
index                      12
id                     338749
name            Swastik Dhaba
city                   Abohar
rating                    0.0
rating_count                0
cost                    200.0
cuisine          North Indian
Name: 12, dtype: object

Encoded Data Shape: (1, 2684)


In [ ]:

df_cleaned = pd.read_csv('/content/drive/MyDrive/Swiggy_Recommendation/cleaned_data.csv')

encoded_sparse = sparse.load_npz("/content/drive/MyDrive/Swiggy_Recommendation/encoded_data_sparse.npz")

row_idx = 100  # Index to check

# Check the original row (including 'id')
print("Original Data (cleaned_data.csv) at index:")
print(df_cleaned.iloc[row_idx])


# Check the corresponding encoded row (it will be a sparse vector)
print("\nEncoded Data at index:")
print(encoded_sparse[row_idx])

# If you want to see the non-zero values in the encoded row:
print("\nNon-zero values in the encoded row:")
print(encoded_sparse[row_idx].toarray())


Original Data (cleaned_data.csv) at index:
level_0                                    100
index                                      100
id                                      133352
name            NEW GANGOUR SWEETS (Adityapur)
city                                 Adityapur
rating                                     4.0
rating_count                                50
cost                                     200.0
cuisine                           South Indian
Name: 100, dtype: object

Encoded Data at index:
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (1, 2687)>
  Coords	Values
  (0, 0)	4.0
  (0, 1)	50.0
  (0, 2)	200.0
  (0, 5)	1.0
  (0, 2423)	1.0

Non-zero values in the encoded row:
[[  4.  50. 200. ...   0.   0.   0.]]


# Recommendation System

* Group similar restaurants based on features like rating, cost, and cuisine.
* When a user inputs preferences, find the closest cluster and suggest restaurants from it.

In [ ]:
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from scipy.sparse import load_npz

# Load Encoded Sparse Data
encoded_sparse = load_npz("/content/drive/MyDrive/Swiggy_Recommendation/encoded_data_sparse.npz")

# Standardize Data
scaler = StandardScaler(with_mean=False)  #
encoded_scaled = scaler.fit_transform(encoded_sparse)

# Train K-Means Model
num_clusters = 10
kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=42, batch_size=256, n_init=10)
kmeans.fit(encoded_scaled)

# Save the Scaler
with open("/content/drive/MyDrive/Swiggy_Recommendation/scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

# Save K-Means Model
with open("/content/drive/MyDrive/Swiggy_Recommendation/kmeans_model.pkl", "wb") as file:
    pickle.dump(kmeans, file)

# **Validation: Load and Test the Model**
with open("/content/drive/MyDrive/Swiggy_Recommendation/kmeans_model.pkl", "rb") as file:
    loaded_kmeans = pickle.load(file)

# Ensure the loaded model predicts correctly
test_labels = loaded_kmeans.predict(encoded_scaled[:5])
print("✅ Model Test Successful. Predicted clusters:", test_labels)

print("✅ K-Means Model & Scaler Saved Successfully!")


✅ Model Test Successful. Predicted clusters: [0 0 0 0 0]
✅ K-Means Model & Scaler Saved Successfully!


 # Result Mapping

 * The Goal is to map these cluster labels back to the original dataset (cleaned_data.csv) so that we can understand which restaurants belong to which cluster.

In [ ]:
import pandas as pd
import pickle
from scipy.sparse import load_npz

#  Load original cleaned dataset
df_cleaned = pd.read_csv("/content/drive/MyDrive/Swiggy_Recommendation/cleaned_data.csv")

#  Load the encoded sparse data
encoded_sparse = load_npz("/content/drive/MyDrive/Swiggy_Recommendation/encoded_data_sparse.npz")

#  Load the scaler & apply transformation
with open("/content/drive/MyDrive/Swiggy_Recommendation/scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

encoded_scaled = scaler.transform(encoded_sparse)

#  Load the trained K-Means model
with open("/content/drive/MyDrive/Swiggy_Recommendation/kmeans_model.pkl", "rb") as file:
    kmeans = pickle.load(file)

#  Predict cluster labels
df_cleaned["cluster"] = kmeans.predict(encoded_scaled)

#  Save mapped results
df_cleaned.to_csv("/content/drive/MyDrive/Swiggy_Recommendation/cleaned_data_with_clusters.csv", index=False)

print("✅ Cluster labels successfully mapped & saved in cleaned_data_with_clusters.csv")


✅ Cluster labels successfully mapped & saved in cleaned_data_with_clusters.csv


In [ ]:
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from scipy.sparse import load_npz

#  Load Encoded Sparse Data
encoded_sparse = load_npz("/content/drive/MyDrive/Swiggy_Recommendation/encoded_data_sparse.npz")

#  Standardize Data
scaler = StandardScaler(with_mean=False)  # With sparse matrix, disable mean centering
encoded_scaled = scaler.fit_transform(encoded_sparse)

#  Train K-Means Model
kmeans = MiniBatchKMeans(n_clusters=10, random_state=42, batch_size=256, n_init=10)  # Adjust clusters if needed
kmeans.fit(encoded_scaled)

#  Save the Scaler
with open("/content/drive/MyDrive/Swiggy_Recommendation/scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

#  Save K-Means Model
with open("/content/drive/MyDrive/Swiggy_Recommendation/kmeans_model.pkl", "wb") as file:
    pickle.dump(kmeans, file)

print("✅ K-Means Model & Scaler Saved Successfully!")


✅ K-Means Model & Scaler Saved Successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Streamlit

🔹 **Swiggy Restaurant Recommendation System (Streamlit)**

📌 Interactive UI: Users select city, cuisine, rating, and cost preferences.

📌 Personalized Recommendations: Filters restaurants based on user input.

📌 Pagination Support: Loads results in pages for better user experience.

📌 Real-Time Updates: Recommendations update instantly on filter changes.

📌 Lottie Animations: Enhances user engagement with animations.

📌 Efficient Data Handling: Uses preprocessed clustering for fast recommendations.

In [ ]:
!pip install pyngrok
!npm install localtunnel
!pip install ngrok
!pip install streamlit

⠙⠹⠸⠼⠴⠦⠧⠇
up to date, audited 23 packages in 2s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠏Requirement already satisfied: ngrok in /usr/local/lib/python3.11/dist-packages (1.4.0)


In [ ]:

# Set your ngrok auth token
from pyngrok import ngrok
ngrok.set_auth_token("2qYh3WXfLyTfZiBiskvg0VXMXds_4bHusVSb8aaK2WHaf6HKz")


# Start ngrok tunnel
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run work.py &>/dev/null&

Streamlit app is live at: NgrokTunnel: "https://74dd-34-125-142-9.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import streamlit as st
import pandas as pd

# Load Cleaned Data with Clusters
df_clustered = pd.read_csv("/content/drive/MyDrive/Swiggy_Recommendation/cleaned_data_with_clusters.csv")

# Cost range mapping
cost_ranges = {
    "All": (df_clustered["cost"].min(), df_clustered["cost"].max()),
    "100 - 250": (100, 250),
    "250 - 500": (250, 500),
    "500 - 1000": (500, 1000),
    "1000 - 2000": (1000, 2000),
    "2000 - 3500": (2000, 3500),
    "3500 - 5000": (3500, 5000)
}

# Initialize session state
if "show_animation" not in st.session_state:
    st.session_state.show_animation = True
if "recommendations" not in st.session_state:
    st.session_state.recommendations = pd.DataFrame()
if "page" not in st.session_state:
    st.session_state.page = 1
if "has_next_page" not in st.session_state:
    st.session_state.has_next_page = False

# Recommendation function
def recommend_restaurants(city, cuisine, min_rating, cost_range, page=1, per_page=10):
    min_cost, max_cost = cost_ranges[cost_range]
    filtered_df = df_clustered[df_clustered["city"].str.lower().str.strip() == city.lower().strip()]

    # Filter cuisines
    if cuisine != "All":
        user_cuisines = {c.strip().lower() for c in cuisine.split(",")}
        filtered_df = filtered_df[filtered_df["cuisine"].str.lower().apply(
            lambda x: bool(user_cuisines & set(x.split(",")))
        )]

    # Filter rating
    if min_rating != "All":
        min_rating = float(min_rating)
        filtered_df = filtered_df[filtered_df["rating"] >= min_rating]

    # Filter cost
    filtered_df = filtered_df[filtered_df["cost"].between(min_cost, max_cost)]

    # Handle empty results
    if filtered_df.empty:
        return pd.DataFrame(), False

    start_idx = (page - 1) * per_page
    end_idx = start_idx + per_page
    has_next_page = end_idx < len(filtered_df)

    return filtered_df.iloc[start_idx:end_idx][["name", "city", "cuisine", "rating", "cost"]], has_next_page

# UI Title
st.markdown("<h1 style='color:#FF5733;'>🍽️ Swiggy Restaurant Recommendation</h1>", unsafe_allow_html=True)

# Display Animation (Only if no results yet)
if st.session_state.show_animation:
    st.markdown(
        """
        <div style="display: flex; justify-content: center; align-items: center; height: 50vh;">
            <iframe src="https://lottie.host/embed/c6c04388-a526-48cd-8484-e51f9fc1e192/CXwrR353NB.lottie"
                width="400" height="400" style="border:none;"></iframe>
        </div>
        """,
        unsafe_allow_html=True
    )

# Sidebar Inputs
st.sidebar.markdown(f'<iframe src="https://lottie.host/embed/3cb37d99-9712-4ae9-9888-a3dc902054e0/zlFLXpDxPz.lottie" width="300" height="150" style="border:none;"></iframe>', unsafe_allow_html=True)
st.sidebar.markdown("<h2 style='color:#FF5733;'>🍽️ Select Your Preferences</h2>", unsafe_allow_html=True)
selected_city = st.sidebar.selectbox("📍 Select a City", sorted(df_clustered["city"].unique()))
available_cuisines = sorted(df_clustered[df_clustered["city"] == selected_city]["cuisine"].unique())
available_cuisines.insert(0, "All")
selected_cuisine = st.sidebar.selectbox("🍛 Select a Cuisine", available_cuisines)
min_rating = st.sidebar.selectbox("⭐ Minimum Rating", ["All", 0, 1, 2, 3, 4, 5], index=1)
selected_cost_range = st.sidebar.selectbox("💰 Select Cost Range", list(cost_ranges.keys()))

# Handle Recommendations
if st.sidebar.button("🔍 Get Recommendations"):
    st.session_state.show_animation = False
    st.session_state.page = 1
    st.session_state.recommendations, st.session_state.has_next_page = recommend_restaurants(
        selected_city, selected_cuisine, min_rating, selected_cost_range, st.session_state.page
    )
    st.rerun()

# Display Recommendations
if not st.session_state.recommendations.empty:
    st.write(f"### 🔹 Recommended Restaurants in {selected_city} - {selected_cuisine}")
    for _, row in st.session_state.recommendations.iterrows():
        st.markdown(f"""
            <div style="border:1px solid #ddd; padding:10px; margin-bottom:10px; border-radius:10px; background:#FFF;">
                <h4>🍴 {row['name']}</h4>
                <p>📍 {row['city']} | 🍛 {row['cuisine']}</p>
                <p>⭐ Rating: {row['rating']} | 💰 Cost: {row['cost']} Rs</p>
            </div>
        """, unsafe_allow_html=True)

    # Pagination Controls
    col1, col2 = st.columns([1, 1])
    with col1:
        if st.session_state.page > 1 and st.button("⬅️ Previous Page"):
            st.session_state.page -= 1
            st.session_state.recommendations, st.session_state.has_next_page = recommend_restaurants(
                selected_city, selected_cuisine, min_rating, selected_cost_range, st.session_state.page
            )
            st.rerun()

    with col2:
        if st.session_state.has_next_page and st.button("Next Page ➡️"):
            st.session_state.page += 1
            st.session_state.recommendations, st.session_state.has_next_page = recommend_restaurants(
                selected_city, selected_cuisine, min_rating, selected_cost_range, st.session_state.page
            )
            st.rerun()

# Show no results message properly
elif not st.session_state.show_animation:
    st.write("🚫 Oops! No Matches Found. Try tweaking the filters! 🍽️🔍")


2025-03-27 06:08:39.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 06:08:39.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 06:08:39.571 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 06:08:39.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 06:08:39.579 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 06:08:39.582 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 06:08:39.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 06:08:39.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar